In [ ]:
import os
from tqdm import tqdm
import pandas as pd
import sys
import time
import logging
import numpy as np
import matplotlib.pyplot as plt
import cv2
import re
from sklearn.neighbors import NearestNeighbors
from pathlib import Path 

In [ ]:
def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(data, key=alphanum_key)

In [ ]:
openpose_path = '/Users/andreibirladeanu/Documents/Data/meal_openpose/1043_meal/'

In [ ]:
poses = [x for x in sorted_alphanumeric(os.listdir(openpose_path))]
#poses = [int(x.split('.')[0].split("_")[1]) for x in poses]

In [ ]:
human_no = []
for frame in sorted_alphanumeric(os.listdir(openpose_path)):
    data = pd.read_csv(openpose_path + frame)
       # print(len(data.columns)-1)
    human_no.append((len(data.columns)-1)/3)
human_no

In [ ]:
human_no.count(3)

In [ ]:
## frames
def return_frames(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    count = 0
    while(cap.isOpened()):
            ret_val, image = cap.read()
            if ret_val == False:
                break
            if count> 100 and count <200:
                frames.append(image)
            count = count+1
                
    return (frames)
frames = return_frames('/Users/andreibirladeanu/Documents/Data/meal_videos/1043_meal.mp4')

In [ ]:
cap = cv2.VideoCapture('/Users/andreibirladeanu/Documents/Data/meal_videos/1043_meal.mp4')
count = 0
while(cap.isOpened()):
    ret_val, image = cap.read()
    if ret_val == False:
                break
    count += 1
    if count in frames_3:
        cv2.imshow('human', image[round(np.nanmin(new_skeleton3[count][0])):round(np.nanmax(new_skeleton3[count][0])),
                       round(np.nanmin(new_skeleton3[count][1])):round(np.nanmax(new_skeleton3[count][1]))])
        cv2.waitKey(0)
    
    

In [ ]:
skeletons = []
for frame in sorted_alphanumeric(os.listdir(openpose_path)):
    data = pd.read_csv(openpose_path + frame, encoding='latin1')
    data = data.fillna(0)
    if (len(data.columns)-1)/3 == 1:
        skeletons.append({'skeleton1':[np.array(data['x_1']), np.array(data['y_1']), np.array(data['prob_1'])]})
    elif (len(data.columns)-1)/3 == 2:
        skeletons.append({'skeleton1':[np.array(data['x_1']), np.array(data['y_1']),  np.array(data['prob_1'])],'skeleton2':[np.array(data['x_2']), np.array(data['y_2']),
                                                                                                                             np.array(data['prob_2'])]})
    elif (len(data.columns)-1)/3 == 3:
        skeletons.append({'skeleton1':[np.array(data['x_1']), np.array(data['y_1']),  np.array(data['prob_1'])],'skeleton2':[np.array(data['x_2']), np.array(data['y_2']),
                             np.array(data['prob_2'])],
                          'skeleton3':[np.array(data['x_3']), np.array(data['y_3']),  np.array(data['prob_3'])]})                                  


In [ ]:
len(skeletons)


In [ ]:
new_skeletons1

In [ ]:
neigh = NearestNeighbors(n_neighbors=1, algorithm = 'brute', metric='euclidean')
new_skeleton1 = []
new_skeleton2 = []
hooman1 = []
counter=0
hooman2_frames = []
for i in range(len(skeletons)): 
    if i == 0 and len(skeletons[i])==1:   # we start in the case where only one skeleton is available
        new_skeleton1.append(skeletons[i]['skeleton1'])
        continue
    if i==0 and len(skeletons[i])==2: ## if both skeletons are available 
        new_skeleton1.append(skeletons[i]['skeleton1'])
        new_skeleton2.append(skeletons[i]['skeleton2'])
        hooman2_frames.append(i)
        continue
    ## the first frame will meet either of these 2 conditions above and then loop will carry on to the next frame
    if len(skeletons[i])==1 and len(skeletons[i-1])==1:
        neigh.fit([[new_skeleton1[-1][0][1],  new_skeleton1[-1][1][1]]])
        dist, nn = neigh.kneighbors([[skeletons[i]['skeleton1'][0][1],  skeletons[i]['skeleton1'][1][1]]], 
                                    return_distance = True)
        if dist[0][0] < 10:
            new_skeleton1.append(skeletons[i]['skeleton1'])


    if len(skeletons[i])==2 and len(skeletons[i-1])==1:
        neigh.fit([[new_skeleton1[-1][0][1], new_skeleton1[-1][1][1]]])
        dist_1, nn_1 = neigh.kneighbors([[skeletons[i]['skeleton1'][0][1],  skeletons[i]['skeleton1'][1][1]]], 
                                    return_distance = True)
        dist_2, nn_2 =  neigh.kneighbors([[skeletons[i]['skeleton2'][0][1],  skeletons[i]['skeleton2'][1][1]]], 
                                    return_distance = True)

        if dist_1 < dist_2 and dist_1 <10:
            new_skeleton1.append(skeletons[i]['skeleton1'])
            new_skeleton2.append(skeletons[i]['skeleton2'])
            hooman2_frames.append(i)
        elif dist_2 < dist_1 and dist_2 <10:
            new_skeleton1.append(skeletons[i]['skeleton2'])
            new_skeleton2.append(skeletons[i]['skeleton1'])
            hooman2_frames.append(i)



    if len(skeletons[i])==1 and len(skeletons[i-1])==2:
        neigh.fit([[new_skeleton1[-1][0][1], new_skeleton1[-1][1][1]], 
                [new_skeleton2[-1][0][1], new_skeleton2[-1][1][1]]])
        dist, nn = neigh.kneighbors([[skeletons[i]['skeleton1'][0][1],  skeletons[i]['skeleton1'][1][1]]], 
                                    return_distance = True)
        if nn[0][0] ==0 and dist[0][0] < 10:
            new_skeleton1.append(skeletons[i]['skeleton1'])
        elif nn[0][0] ==1 and dist[0][0] < 10:
            new_skeleton2.append(skeletons[i]['skeleton1'])


    if len(skeletons[i])==2 and len(skeletons[i-1])==2:
        neigh.fit([[new_skeleton1[-1][0][1], new_skeleton1[-1][1][1]], 
                [new_skeleton2[-1][0][1], new_skeleton2[-1][1][1]]])
    
        
        dist_1, nn_1 = neigh.kneighbors([[skeletons[i]['skeleton1'][0][1],  skeletons[i]['skeleton1'][1][1]]], 
                                    n_neighbors=1, return_distance = True)
  
        dist_2, nn_2 = neigh.kneighbors([[skeletons[i]['skeleton2'][0][1],  skeletons[i]['skeleton2'][1][1]]], 
                                    return_distance = True)

        if nn_1[0][0] == 0 and dist_1[0][0] < 15:
            new_skeleton1.append(skeletons[i]['skeleton1'])
            if dist_2[0][0] <15: ## making sure the other person hasn't suddenly switched to something/someone else
                new_skeleton2.append(skeletons[i]['skeleton2'])
                hooman2_frames.append(i)
        elif nn_1[0][0] == 1 and dist_1[0][0] < 15:
            new_skeleton2.append(skeletons[i]['skeleton1'])
            if dist_2[0][0] <15:
                new_skeleton1.append(skeletons[i]['skeleton2'])
                hooman2_frames.append(i)
        else:
            print("2 and 2", i)

    counter +=1

    #hooman1.append(frames[i][round(np.nanmin(new_skeleton1[i][1])):round(np.nanmax(new_skeleton1[i][1])),
     #                   round(np.nanmin(new_skeleton1[i][0])):round(np.nanmax(new_skeleton1[i][0]))])            

        

In [ ]:

for hooman in hooman1:
    cv2.imshow('human1', hooman)
    cv2.waitKey(0) 

In [ ]:
hooman2=[]
for ind, skel in zip(hooman2_frames, new_skeleton2):
    hooman2.append(frames[ind][round(np.nanmin(skel[1])):round(np.nanmax(skel[1])),
                        round(np.nanmin(skel[0])):round(np.nanmax(skel[0]))])

In [ ]:
for hooman in hooman2:
    cv2.imshow('human2', hooman)
    cv2.waitKey(0) 

In [ ]:
len(frames)

In [ ]:
## saving output
output_path = '/Users/andrei-macpro/Documents/Data/1042_meal_skeletons/child/'
for hooman in range(len(hooman2)):
    cv2.imwrite(output_path + 'frame '+ str(hooman)+'.png', hooman2[hooman])

In [ ]:
len(new_skeleton1)

In [ ]:
## pop it out? 



In [ ]:
skeletons[0]['skeleton1']

In [ ]:
### new pop rule
### extracting Skeleton 1

neigh = NearestNeighbors(n_neighbors=1, algorithm = 'brute', metric='euclidean')
new_skeleton1 = []
hooman1 = []
counter=0
hooman2_frames = []
main_kpt = 1
fallback = 0
frames_1=[]
for i in range(len(skeletons)): 
        if i==0:
            new_skeleton1.append(skeletons[i]['skeleton1'])
            frames_1.append(i+1)
            skeletons[i].pop('skeleton1')
            continue
        if len(skeletons[i])==1:
            if new_skeleton1[-1][0][main_kpt] and new_skeleton1[-1][1][main_kpt] and \
                skeletons[i]['skeleton1'][0][main_kpt] and  skeletons[i]['skeleton1'][1][main_kpt] > 0.0:
                neigh.fit([[new_skeleton1[-1][0][main_kpt],  new_skeleton1[-1][1][main_kpt]]])
                neigh.fit([[new_skeleton1[-1][0][main_kpt],  new_skeleton1[-1][1][main_kpt]]])
                dist, nn = neigh.kneighbors([[skeletons[i]['skeleton1'][0][main_kpt],  skeletons[i]['skeleton1'][1][main_kpt]]], 
                                        return_distance = True)
                
                if dist[0][0] < 50:
                    new_skeleton1.append(skeletons[i]['skeleton1'])
                    frames_1.append(i+1)
                    skeletons[i].pop('skeleton1')
            else:
                neigh.fit([[new_skeleton1[-1][0][fallback],  new_skeleton1[-1][1][fallback]]])
                neigh.fit([[new_skeleton1[-1][0][fallback],  new_skeleton1[-1][1][fallback]]])
                dist, nn = neigh.kneighbors([[skeletons[i]['skeleton1'][0][fallback],  skeletons[i]['skeleton1'][1][fallback]]], 
                                        return_distance = True)
                
                if dist[0][0] < 50:
                    new_skeleton1.append(skeletons[i]['skeleton1'])
                    frames_1.append(i+1)
                    skeletons[i].pop('skeleton1')
            
        elif len(skeletons[i])==2:
            if new_skeleton1[-1][0][main_kpt] and new_skeleton1[-1][1][main_kpt] and \
           skeletons[i]['skeleton1'][0][main_kpt] and  skeletons[i]['skeleton1'][1][main_kpt] and \
           skeletons[i]['skeleton2'][0][main_kpt] and  skeletons[i]['skeleton2'][1][main_kpt]> 0.0:
            
                neigh.fit([[new_skeleton1[-1][0][main_kpt],  new_skeleton1[-1][1][main_kpt]]])
                dist_0, nn_0 = neigh.kneighbors([[skeletons[i]['skeleton1'][0][main_kpt],  skeletons[i]['skeleton1'][1][main_kpt]]], 
                                        return_distance = True)
                dist_1, nn_1 = neigh.kneighbors([[skeletons[i]['skeleton2'][0][main_kpt],  skeletons[i]['skeleton2'][1][main_kpt]]], 
                                        return_distance = True)
                if dist_0 < dist_1 and dist_0 <50:
                    new_skeleton1.append(skeletons[i]['skeleton1'])
                    frames_1.append(i+1)
                    skeletons[i].pop('skeleton1')
                elif dist_0 > dist_1 and dist_1 <50:
                    new_skeleton1.append(skeletons[i]['skeleton2'])
                    frames_1.append(i+1)
                    skeletons[i].pop('skeleton2')
            else:
                neigh.fit([[new_skeleton1[-1][0][fallback],  new_skeleton1[-1][1][fallback]]])
                dist_0, nn_0 = neigh.kneighbors([[skeletons[i]['skeleton1'][0][fallback],  skeletons[i]['skeleton1'][1][fallback]]], 
                                        return_distance = True)
                dist_1, nn_1 = neigh.kneighbors([[skeletons[i]['skeleton2'][0][fallback],  skeletons[i]['skeleton2'][1][fallback]]], 
                                        return_distance = True)
                if dist_0 < dist_1 and dist_0 <50:
                    new_skeleton1.append(skeletons[i]['skeleton1'])
                    frames_1.append(i+1)
                    skeletons[i].pop('skeleton1')
                elif dist_0 > dist_1 and dist_1 <50:
                    new_skeleton1.append(skeletons[i]['skeleton2'])
                    frames_1.append(i+1)
                    skeletons[i].pop('skeleton2')
        
    
     
        elif len(skeletons[i])==3:
            if new_skeleton1[-1][0][main_kpt] and new_skeleton1[-1][1][main_kpt] and \
           skeletons[i]['skeleton1'][0][main_kpt] and  skeletons[i]['skeleton1'][1][main_kpt] and \
           skeletons[i]['skeleton2'][0][main_kpt] and  skeletons[i]['skeleton2'][1][main_kpt] and \
            skeletons[i]['skeleton3'][0][main_kpt] and  skeletons[i]['skeleton3'][1][main_kpt] > 0.0:
                neigh.fit([[new_skeleton1[-1][0][main_kpt],  new_skeleton1[-1][1][main_kpt]]])
                dist_0, nn_0 = neigh.kneighbors([[skeletons[i]['skeleton1'][0][main_kpt],  skeletons[i]['skeleton1'][1][main_kpt]]], 
                                        return_distance = True)
                dist_1, nn_1 = neigh.kneighbors([[skeletons[i]['skeleton2'][0][main_kpt],  skeletons[i]['skeleton2'][1][main_kpt]]], 
                                        return_distance = True)
                dist_2, nn_2 = neigh.kneighbors([[skeletons[i]['skeleton3'][0][main_kpt],  skeletons[i]['skeleton3'][1][main_kpt]]], 
                                        return_distance = True)
                if dist_0 < dist_1 < dist_2 and dist_0<50:
                    new_skeleton1.append(skeletons[i]['skeleton1'])
                    frames_1.append(i+1)
                    skeletons[i].pop('skeleton1')
                elif dist_1 < dist_0 < dist_2 and dist_1<50:
                    new_skeleton1.append(skeletons[i]['skeleton2'])
                    frames_1.append(i+1)
                    skeletons[i].pop('skeleton2')
                elif dist_2 < dist_1 < dist_0 and dist_2<50:
                    new_skeleton1.append(skeletons[i]['skeleton3'])
                    frames_1.append(i+1)
                    skeletons[i].pop('skeleton3')
            else:
                neigh.fit([[new_skeleton1[-1][0][fallback],  new_skeleton1[-1][1][fallback]]])
                dist_0, nn_0 = neigh.kneighbors([[skeletons[i]['skeleton1'][0][fallback],  skeletons[i]['skeleton1'][1][fallback]]], 
                                        return_distance = True)
                dist_1, nn_1 = neigh.kneighbors([[skeletons[i]['skeleton2'][0][fallback],  skeletons[i]['skeleton2'][1][fallback]]], 
                                        return_distance = True)
                dist_2, nn_2 = neigh.kneighbors([[skeletons[i]['skeleton3'][0][fallback],  skeletons[i]['skeleton3'][1][fallback]]], 
                                        return_distance = True)
                if dist_0 < dist_1 < dist_2 and dist_0<50:
                    new_skeleton1.append(skeletons[i]['skeleton1'])
                    frames_1.append(i+1)
                    skeletons[i].pop('skeleton1')
                elif dist_1 < dist_0 < dist_2 and dist_1<50:
                    new_skeleton1.append(skeletons[i]['skeleton2'])
                    frames_1.append(i+1)
                    skeletons[i].pop('skeleton2')
                elif dist_2 < dist_1 < dist_0 and dist_2<50:
                    new_skeleton1.append(skeletons[i]['skeleton3'])
                    frames_1.append(i+1)
                    skeletons[i].pop('skeleton3')

            


In [ ]:
len(new_skeleton1)

In [ ]:
### new pop rule
## extracting skeleton 2

neigh = NearestNeighbors(n_neighbors=1, algorithm = 'brute', metric='euclidean')
new_skeleton2 = []
hooman1 = []
counter=0
frames_2 = []
for i in range(len(skeletons)): 
    if skeletons[i] == {}:
        continue
    if len(skeletons[i])==1:
        new_skeleton2.append(list(skeletons[i].items())[0][1])
        frames_2.append(i+1)
        if list(skeletons[i].items())[0][0] == 'skeleton2':
            skeletons[i].pop('skeleton2')
        else:
            skeletons[i].pop('skeleton1')

    elif len(skeletons[i])==2 and len(skeletons[i-1])==1:
        if new_skeleton2[-1][0][main_kpt] and new_skeleton2[-1][1][main_kpt] and \
         skeletons[i][0][1][0][main_kpt] and skeletons[i][0][1][1][main_kpt] and \
         skeletons[i][1][1][0][main_kpt] and skeletons[i][1][1][1][main_kpt] > 0.0:
            neigh.fit([[new_skeleton2[-1][0][main_kpt],  new_skeleton2[-1][1][main_kpt]]])
            dist_0, nn_0 = neigh.kneighbors([[skeletons[i][0][1][0][main_kpt],  skeletons[i][0][1][1][main_kpt]]], 
                                    return_distance = True)
            dist_1, nn_1 = neigh.kneighbors([[skeletons[i][1][1][0][main_kpt],  skeletons[i][1][1][1][main_kpt]]], 
                                    return_distance = True)
            if dist_0 < dist_1 and dist_0 <50:
                new_skeleton2.append(list(skeletons[i].items())[0][1])
                frames_2.append(i+1)
                if list(skeletons[i].items())[0][0] == 'skeleton2':
                    skeletons[i].pop('skeleton2')
                else:
                    skeletons[i].pop('skeleton1')
            elif dist_0 > dist_1 and dist_1 <50:
                new_skeleton2.append(list(skeletons[i].items())[1][1])
                frames_2.append(i+1)
                if list(skeletons[i].items())[0][0] == 'skeleton2':
                    skeletons[i].pop('skeleton2')
                else:
                    skeletons[i].pop('skeleton1')
        else: 
            neigh.fit([[new_skeleton2[-1][0][fallback],  new_skeleton2[-1][1][fallback]]])
            dist_0, nn_0 = neigh.kneighbors([[skeletons[i][0][1][0][fallback],  skeletons[i][0][1][1][fallback]]], 
                                    return_distance = True)
            dist_1, nn_1 = neigh.kneighbors([[skeletons[i][1][1][0][fallback],  skeletons[i][1][1][1][fallback]]], 
                                    return_distance = True)
            if dist_0 < dist_1 and dist_0 <50:
                new_skeleton2.append(list(skeletons[i].items())[0][1])
                frames_2.append(i+1)
                if list(skeletons[i].items())[0][0] == 'skeleton2':
                    skeletons[i].pop('skeleton2')
                else:
                    skeletons[i].pop('skeleton1')
            elif dist_0 > dist_1 and dist_1 <50:
                new_skeleton2.append(list(skeletons[i].items())[1][1])
                frames_2.append(i+1)
                if list(skeletons[i].items())[0][0] == 'skeleton2':
                    skeletons[i].pop('skeleton2')
                else:
                    skeletons[i].pop('skeleton1')

        


In [ ]:
# skel 3 is what should be left 
new_skeleton3 = []
hooman1 = []
counter=0
frames_3 = []
for i in range(len(skeletons)): 
    if skeletons[i] == {}:
        continue
    if len(skeletons[i]) != {}:
        new_skeleton3.append(list(skeletons[i].items())[0][1])
        frames_3.append(i-1)

In [ ]:
len(new_skeleton3)

In [ ]:
len(skeletons)

In [ ]:
skeletons[20].keys() == 'skeleton2'

In [ ]:
skeletons[20].keys() == (['skeleton2'])

In [ ]:
list(skeletons[20].items())[0][1]

In [ ]:
skeletons[20][0]

In [ ]:
np.savetxt('test.csv', np.transpose(new_skeleton1[0]), delimiter=',')

In [ ]:
new_skeleton1

In [ ]:
## save skeleton to separate csv 
for skeleton, frame in zip(new_skeletonx, frames_x):
    

In [ ]:
pd.DataFrame(np.transpose(new_skeleton1[0]),  columns=['x', 'y', 'p'])

In [ ]:
np.transpose(new_skeleton1[0])

In [ ]:
### from here it starts

In [ ]:
path_openpose = '/Users/andreibirladeanu/Documents/Data/meal_openpose/'

coords = [file for file in sorted(os.listdir(path_openpose)) if file[0] !="."]
print(coords)

In [ ]:
thr = 20
for coord in tqdm(coords):
    new_path = path_openpose+coord
    os.chdir(new_path)
    skeletons = []
    for frame in sorted_alphanumeric(os.listdir(new_path)):
        if frame[0] != ".":
            data = pd.read_csv(frame)
            data = data.fillna(0)
        else:
            continue
        if (len(data.columns)-1)/3 == 1:
            skeletons.append({'skeleton1':[np.array(data['x_1']), np.array(data['y_1']), np.array(data['prob_1'])]})
        elif (len(data.columns)-1)/3 == 2:
            skeletons.append({'skeleton1':[np.array(data['x_1']), np.array(data['y_1']),  np.array(data['prob_1'])],'skeleton2':[np.array(data['x_2']), np.array(data['y_2']),
                                                                                                                             np.array(data['prob_2'])]})
        elif (len(data.columns)-1)/3 == 3:
            skeletons.append({'skeleton1':[np.array(data['x_1']), np.array(data['y_1']),  np.array(data['prob_1'])],'skeleton2':[np.array(data['x_2']), np.array(data['y_2']),
                             np.array(data['prob_2'])],
                          'skeleton3':[np.array(data['x_3']), np.array(data['y_3']),  np.array(data['prob_3'])]})  
    ###extracting person 1 
    neigh = NearestNeighbors(n_neighbors=1, algorithm = 'brute', metric='euclidean')
    new_skeleton1 = []
    main_kpt = 1
    fallback = 0
    frames_1=[]
    for i in range(len(skeletons)): 
        if i==0:
            new_skeleton1.append(skeletons[i]['skeleton1'])
            frames_1.append(i+1)
            skeletons[i].pop('skeleton1')
            continue
        if len(skeletons[i])==1:
            if new_skeleton1[-1][0][main_kpt] and new_skeleton1[-1][1][main_kpt] and \
                skeletons[i]['skeleton1'][0][main_kpt] and  skeletons[i]['skeleton1'][1][main_kpt] > 0.0:
                neigh.fit([[new_skeleton1[-1][0][main_kpt],  new_skeleton1[-1][1][main_kpt]]])
                neigh.fit([[new_skeleton1[-1][0][main_kpt],  new_skeleton1[-1][1][main_kpt]]])
                dist, nn = neigh.kneighbors([[skeletons[i]['skeleton1'][0][main_kpt],  skeletons[i]['skeleton1'][1][main_kpt]]], 
                                        return_distance = True)
                
                if dist[0][0] < thr:
                    new_skeleton1.append(skeletons[i]['skeleton1'])
                    frames_1.append(i+1)
                    skeletons[i].pop('skeleton1')
            else:
                neigh.fit([[new_skeleton1[-1][0][fallback],  new_skeleton1[-1][1][fallback]]])
                neigh.fit([[new_skeleton1[-1][0][fallback],  new_skeleton1[-1][1][fallback]]])
                dist, nn = neigh.kneighbors([[skeletons[i]['skeleton1'][0][fallback],  skeletons[i]['skeleton1'][1][fallback]]], 
                                        return_distance = True)
                
                if dist[0][0] < thr:
                    new_skeleton1.append(skeletons[i]['skeleton1'])
                    frames_1.append(i+1)
                    skeletons[i].pop('skeleton1')
            
        elif len(skeletons[i])==2:
            if new_skeleton1[-1][0][main_kpt] and new_skeleton1[-1][1][main_kpt] and \
           skeletons[i]['skeleton1'][0][main_kpt] and  skeletons[i]['skeleton1'][1][main_kpt] and \
           skeletons[i]['skeleton2'][0][main_kpt] and  skeletons[i]['skeleton2'][1][main_kpt]> 0.0:
            
                neigh.fit([[new_skeleton1[-1][0][main_kpt],  new_skeleton1[-1][1][main_kpt]]])
                dist_0, nn_0 = neigh.kneighbors([[skeletons[i]['skeleton1'][0][main_kpt],  skeletons[i]['skeleton1'][1][main_kpt]]], 
                                        return_distance = True)
                dist_1, nn_1 = neigh.kneighbors([[skeletons[i]['skeleton2'][0][main_kpt],  skeletons[i]['skeleton2'][1][main_kpt]]], 
                                        return_distance = True)
                if dist_0 < dist_1 and dist_0 <thr:
                    new_skeleton1.append(skeletons[i]['skeleton1'])
                    frames_1.append(i+1)
                    skeletons[i].pop('skeleton1')
                elif dist_0 > dist_1 and dist_1 <thr:
                    new_skeleton1.append(skeletons[i]['skeleton2'])
                    frames_1.append(i+1)
                    skeletons[i].pop('skeleton2')
            else:
                neigh.fit([[new_skeleton1[-1][0][fallback],  new_skeleton1[-1][1][fallback]]])
                dist_0, nn_0 = neigh.kneighbors([[skeletons[i]['skeleton1'][0][fallback],  skeletons[i]['skeleton1'][1][fallback]]], 
                                        return_distance = True)
                dist_1, nn_1 = neigh.kneighbors([[skeletons[i]['skeleton2'][0][fallback],  skeletons[i]['skeleton2'][1][fallback]]], 
                                        return_distance = True)
                if dist_0 < dist_1 and dist_0 <thr:
                    new_skeleton1.append(skeletons[i]['skeleton1'])
                    frames_1.append(i+1)
                    skeletons[i].pop('skeleton1')
                elif dist_0 > dist_1 and dist_1 <thr:
                    new_skeleton1.append(skeletons[i]['skeleton2'])
                    frames_1.append(i+1)
                    skeletons[i].pop('skeleton2')
                
        elif len(skeletons[i])==3:
            if new_skeleton1[-1][0][main_kpt] and new_skeleton1[-1][1][main_kpt] and \
           skeletons[i]['skeleton1'][0][main_kpt] and  skeletons[i]['skeleton1'][1][main_kpt] and \
           skeletons[i]['skeleton2'][0][main_kpt] and  skeletons[i]['skeleton2'][1][main_kpt] and \
            skeletons[i]['skeleton3'][0][main_kpt] and  skeletons[i]['skeleton3'][1][main_kpt] > 0.0:
                neigh.fit([[new_skeleton1[-1][0][main_kpt],  new_skeleton1[-1][1][main_kpt]]])
                dist_0, nn_0 = neigh.kneighbors([[skeletons[i]['skeleton1'][0][main_kpt],  skeletons[i]['skeleton1'][1][main_kpt]]], 
                                        return_distance = True)
                dist_1, nn_1 = neigh.kneighbors([[skeletons[i]['skeleton2'][0][main_kpt],  skeletons[i]['skeleton2'][1][main_kpt]]], 
                                        return_distance = True)
                dist_2, nn_2 = neigh.kneighbors([[skeletons[i]['skeleton3'][0][main_kpt],  skeletons[i]['skeleton3'][1][main_kpt]]], 
                                        return_distance = True)
                if dist_0 < dist_1 < dist_2 and dist_0<thr:
                    new_skeleton1.append(skeletons[i]['skeleton1'])
                    frames_1.append(i+1)
                    skeletons[i].pop('skeleton1')
                elif dist_1 < dist_0 < dist_2 and dist_1<thr:
                    new_skeleton1.append(skeletons[i]['skeleton2'])
                    frames_1.append(i+1)
                    skeletons[i].pop('skeleton2')
                elif dist_2 < dist_1 < dist_0 and dist_2<thr:
                    new_skeleton1.append(skeletons[i]['skeleton3'])
                    frames_1.append(i+1)
                    skeletons[i].pop('skeleton3')
            else:
                neigh.fit([[new_skeleton1[-1][0][fallback],  new_skeleton1[-1][1][fallback]]])
                dist_0, nn_0 = neigh.kneighbors([[skeletons[i]['skeleton1'][0][fallback],  skeletons[i]['skeleton1'][1][fallback]]], 
                                        return_distance = True)
                dist_1, nn_1 = neigh.kneighbors([[skeletons[i]['skeleton2'][0][fallback],  skeletons[i]['skeleton2'][1][fallback]]], 
                                        return_distance = True)
                dist_2, nn_2 = neigh.kneighbors([[skeletons[i]['skeleton3'][0][fallback],  skeletons[i]['skeleton3'][1][fallback]]], 
                                        return_distance = True)
                if dist_0 < dist_1 < dist_2 and dist_0<thr:
                    new_skeleton1.append(skeletons[i]['skeleton1'])
                    frames_1.append(i+1)
                    skeletons[i].pop('skeleton1')
                elif dist_1 < dist_0 < dist_2 and dist_1<thr:
                    new_skeleton1.append(skeletons[i]['skeleton2'])
                    frames_1.append(i+1)
                    skeletons[i].pop('skeleton2')
                elif dist_2 < dist_1 < dist_0 and dist_2<thr:
                    new_skeleton1.append(skeletons[i]['skeleton3'])
                    frames_1.append(i+1)
    os.makedirs('/Users/andreibirladeanu/Documents/Data/new_meal_openpose/' +coord + '/' + 'skel1/', exist_ok=True)  
    for skeleton, frame_no in zip(new_skeleton1, frames_1):
        out = pd.DataFrame(np.transpose(skeleton),  columns=['x', 'y', 'p'])
        filepath = Path('/Users/andreibirladeanu/Documents/Data/new_meal_openpose/' +coord +'/' + 'skel1/' +str(frame_no) +'.csv')
        #filepath.parent.mkdir(parents=True, exist_ok=True)  
        out.to_csv(filepath)
        
    ### tracking second skeleton 
    neigh = NearestNeighbors(n_neighbors=1, algorithm = 'brute', metric='euclidean')
    new_skeleton2 = []
    frames_2 = []
    for i in range(len(skeletons)): 
        if skeletons[i] == {}:
            continue
        if len(skeletons[i])==1:
            new_skeleton2.append(list(skeletons[i].items())[0][1])
            frames_2.append(i+1)
            if list(skeletons[i].items())[0][0] == 'skeleton2':
                skeletons[i].pop('skeleton2')
            else:
                skeletons[i].pop('skeleton1')
        elif i==0 and len(skeletons[i])==2:
            new_skeleton2.append(list(skeletons[i].items())[0][1])
            frames_2.append(i+1)
            if list(skeletons[i].items())[0][0] == 'skeleton2':
                skeletons[i].pop('skeleton2')
            else:
                skeletons[i].pop('skeleton1')
            
        elif len(skeletons[i])==2 and len(skeletons[i-1])==1:
            print(new_skeleton2)
            if new_skeleton2[-1][0][main_kpt] and new_skeleton2[-1][1][main_kpt] and \
             skeletons[i][0][1][0][main_kpt] and skeletons[i][0][1][1][main_kpt] and \
             skeletons[i][1][1][0][main_kpt] and skeletons[i][1][1][1][main_kpt] > 0.0:
                neigh.fit([[new_skeleton2[-1][0][main_kpt],  new_skeleton2[-1][1][main_kpt]]])
                dist_0, nn_0 = neigh.kneighbors([[skeletons[i][0][1][0][main_kpt],  skeletons[i][0][1][1][main_kpt]]], 
                                        return_distance = True)
                dist_1, nn_1 = neigh.kneighbors([[skeletons[i][1][1][0][main_kpt],  skeletons[i][1][1][1][main_kpt]]], 
                                        return_distance = True)
                if dist_0 < dist_1 and dist_0 <thr:
                    new_skeleton2.append(list(skeletons[i].items())[0][1])
                    frames_2.append(i+1)
                    if list(skeletons[i].items())[0][0] == 'skeleton2':
                        skeletons[i].pop('skeleton2')
                    else:
                        skeletons[i].pop('skeleton1')
                elif dist_0 > dist_1 and dist_1 <thr:
                    new_skeleton2.append(list(skeletons[i].items())[1][1])
                    frames_2.append(i+1)
                    if list(skeletons[i].items())[0][0] == 'skeleton2':
                        skeletons[i].pop('skeleton2')
                    else:
                        skeletons[i].pop('skeleton1')
            else: 
                neigh.fit([[new_skeleton2[-1][0][fallback],  new_skeleton2[-1][1][fallback]]])
                dist_0, nn_0 = neigh.kneighbors([[skeletons[i][0][1][0][fallback],  skeletons[i][0][1][1][fallback]]], 
                                        return_distance = True)
                dist_1, nn_1 = neigh.kneighbors([[skeletons[i][1][1][0][fallback],  skeletons[i][1][1][1][fallback]]], 
                                        return_distance = True)
                if dist_0 < dist_1 and dist_0 <thr:
                    new_skeleton2.append(list(skeletons[i].items())[0][1])
                    frames_2.append(i+1)
                    if list(skeletons[i].items())[0][0] == 'skeleton2':
                        skeletons[i].pop('skeleton2')
                    else:
                        skeletons[i].pop('skeleton1')
                elif dist_0 > dist_1 and dist_1 <thr:
                    new_skeleton2.append(list(skeletons[i].items())[1][1])
                    frames_2.append(i+1)
                    if list(skeletons[i].items())[0][0] == 'skeleton2':
                        skeletons[i].pop('skeleton2')
                    else:
                        skeletons[i].pop('skeleton1') 
    for skeleton, frame_no in zip(new_skeleton2, frames_2):
        out = pd.DataFrame(np.transpose(skeleton),  columns=['x', 'y', 'p'])
        filepath = Path('/Users/andreibirladeanu/Documents/Data/new_meal_openpose/' +coord +'/' + 'skel2/' +str(frame_no) +'.csv')
        filepath.parent.mkdir(parents=True, exist_ok=True)  
        out.to_csv(filepath)
        
        
    ### extracting skeleton 3
    new_skeleton3 = []
    frames_3 = []
    for i in range(len(skeletons)): 
        if skeletons[i] == {}:
            continue
        if len(skeletons[i]) != {}:
            new_skeleton3.append(list(skeletons[i].items())[0][1])
            frames_3.append(i-1)
    for skeleton, frame_no in zip(new_skeleton3, frames_3):
        out = pd.DataFrame(np.transpose(skeleton),  columns=['x', 'y', 'p'])
        filepath = Path('/Users/andreibirladeanu/Documents/Data/new_meal_openpose/' +coord +'/' + 'skel3/' +str(frame_no) +'.csv')
        filepath.parent.mkdir(parents=True, exist_ok=True)  
        out.to_csv(filepath)
    

In [ ]:
print('a')